In [1]:
import os
cwd = os.getcwd()

# protection against running this cell multiple times
assert os.path.dirname(cwd).split('/')[-1] == 'master-thesis','Oops, directory already changed previously as indended. Ignoring...'

# change working directory (if assert passed)
new_cwd = os.path.dirname(cwd) # parent directory
os.chdir(new_cwd)

In [2]:
import os
from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import RNN, train_one_epoch, eval_model, compute_loss

import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np

from config import Settings; settings = Settings()

import wandb

# Parameters

In [3]:
dataset_id = 'diag_only'

# model
grouping = 'ccs'
batch_size=64

# Reproducibility

In [4]:
# Reproducibility
seed = settings.random_seed

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create dataset

In [5]:
dataset_folder = os.path.join(settings.data_base,settings.model_ready_dataset_folder,dataset_id)
print('dataset at',dataset_folder)

dataset at data/model_ready_dataset/diag_only


In [6]:
dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset.json'),grouping)

train_dataset = DiagnosesDataset(os.path.join(dataset_folder,'train_subset.json'),grouping)
val_dataset = DiagnosesDataset(os.path.join(dataset_folder,'val_subset.json'),grouping)
test_dataset = DiagnosesDataset(os.path.join(dataset_folder,'test_subset.json'),grouping)


len(train_dataset)
len(val_dataset)
len(test_dataset)


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

5249

1125

1125

# wandb

In [7]:
wandb.login()

wandb: Currently logged in as: snovaisg (use `wandb login --relogin` to force relogin)


True

## Define data and train conditions

In [8]:
n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

## Define search space

In [11]:
hyperparameters = {
    'hidden_size':[25,50,75],
    'num_layers':[2],
    'lr':[0.01,0.02,0.03],
    'model':['rnn','gru','lstm']
    
}
meta_parameters = {
    'epochs':15
}

params = ParameterGrid(hyperparameters)
print(f'params:',len(params))

#random_params = ParameterSampler(params.param_grid,n_iter=len(params)-1,random_state=231)
#next(iter(random_params))

params: 27


## Run models

In [12]:
for idx,param_set in enumerate(params):
    config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="basic_deterministic_model_tunning", 
        config=config
    )
    
    model = RNN(input_size=input_size,
              hidden_size=config['hidden_size'],
              num_layers=config['num_layers'],
              n_labels=n_labels,
              model=config['model'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    for epoch in range(1,config['epochs']+1):
        log = {}
        
        loss = train_one_epoch(model,train_dataloader,epoch,criterion,optimizer);
        
        wandb.log({'epoch':epoch,'loss':loss})
        
    train_metrics = eval_model(model,train_dataloader,dataset,metrics=['roc','f1'])[1].filter(regex='_adm')
    val_metrics = eval_model(model,val_dataloader,dataset,metrics=['roc','f1'])[1].filter(regex='_adm')
    train_metrics.index = ['train_' + n for n in train_metrics.index]
    val_metrics.index = ['val_' + n for n in val_metrics.index]
        

    log = dict()

    log.update(train_metrics.to_dict())
    log.update(val_metrics.to_dict())
    log.update({'loss':loss})

    wandb.log(log)

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.13024
train_f1_adm,0.1715
train_roc_adm,0.89203
val_f1_adm,0.15828


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.12936
train_f1_adm,0.18032
train_roc_adm,0.89295
val_f1_adm,0.17239


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.13395
train_f1_adm,0.13489
train_roc_adm,0.88217
val_f1_adm,0.13548


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.13225
train_f1_adm,0.1554
train_roc_adm,0.88497
val_f1_adm,0.149


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.12442
train_f1_adm,0.19463
train_roc_adm,0.90139
val_f1_adm,0.18134


...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▃▃▃▂▂▁▁▁▂▂▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.12758
train_f1_adm,0.20117
train_roc_adm,0.89428
val_f1_adm,0.17082


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.10807
train_f1_adm,0.28816
train_roc_adm,0.93349
val_f1_adm,0.18992


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▄▃▃▃▃▃▃▂▂▂▂▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.11088
train_f1_adm,0.26293
train_roc_adm,0.92601
val_f1_adm,0.19708


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▄▄▃▂▂▂▁▂▂▂▁▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.13037
train_f1_adm,0.19461
train_roc_adm,0.88427
val_f1_adm,0.16961


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▄▄▄▄▄▃▃▃▂▂▁▁▁
train_f1_adm,▁
train_roc_adm,▁
val_f1_adm,▁
val_roc_adm,▁
epoch,15
loss,0.11519
train_f1_adm,0.25035
train_roc_adm,0.922
val_f1_adm,0.18938


# best model

- name: silver-salad-21
- lr: 0.01
- model_type = 'lstm'
- num_layers = 1
- hidden_size = 75
- epochs: 15

In [ ]:
hyperparameters = {
    'hidden_size':[25,50,75],
    'num_layers':[2],
    'lr':[0.01,0.02,0.03],
    'model':['rnn','gru','lstm']
    
}

In [13]:
meta_parameters

{'epochs': 15}

In [16]:
param_set = dict(hidden_size=75,num_layers=1,lr=0.01,model_type='lstm',epochs=15,mode='metrics')
wandb.init(
    project="basic_deterministic_model_tunning", 
    config=param_set
)

In [19]:
model = RNN(input_size=input_size,
            hidden_size=param_set['hidden_size'],
            num_layers=param_set['num_layers'],
            n_labels=n_labels,
            model=param_set['model_type'])
optimizer = torch.optim.Adam(model.parameters(), lr=param_set['lr'])
for epoch in range(1,config['epochs']+1):
    loss = train_one_epoch(model,train_dataloader,epoch,criterion,optimizer);

In [ ]:
config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="basic_deterministic_model_tunning", 
        config=config
    )
    
    model = RNN(input_size=input_size,
              hidden_size=config['hidden_size'],
              num_layers=config['num_layers'],
              n_labels=n_labels,
              model=config['model'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    for epoch in range(1,config['epochs']+1):
        log = {}
        
        loss = train_one_epoch(model,train_dataloader,epoch,criterion,optimizer);
        
        wandb.log({'epoch':epoch,'loss':loss})
        
    train_metrics = eval_model(model,train_dataloader,dataset,metrics=['roc','f1'])[1].filter(regex='_adm')
    val_metrics = eval_model(model,val_dataloader,dataset,metrics=['roc','f1'])[1].filter(regex='_adm')
    train_metrics.index = ['train_' + n for n in train_metrics.index]
    val_metrics.index = ['val_' + n for n in val_metrics.index]
        

    log = dict()

    log.update(train_metrics.to_dict())
    log.update(val_metrics.to_dict())
    log.update({'loss':loss})

    wandb.log(log)